In [7]:
import pandas as pd
from sqlalchemy import create_engine
import requests

In [8]:

# URL of the website containing the table
url = "https://www.ndbc.noaa.gov/data/realtime2/"

# Read HTML tables into a list of DataFrames
dfs = pd.read_html(url)

# Select the desired table (e.g., the first one)
df= dfs[0]

df[["ID", "Type"]] =df["Name"].str.split(".", n=1, expand=True)

df = df.iloc[1:, :].dropna(axis=1)
# Print the DataFrame
df.columns


Index(['Name', 'Last modified', 'Size', 'Description', 'ID', 'Type'], dtype='object')

In [61]:
user="surf_app"
pwd="password1"
host="localhost"
db="surfing_project"

con_string=f"postgresql+psycopg2://{user}:{pwd}@{host}/{db}"

engine = create_engine(con_string)


In [62]:
df.to_sql("NewFiles", con=engine, if_exists='replace', index='False')

532

In [60]:
sql = "SELECT * FROM public.\"NewFiles\";"
files = pd.read_sql(sql, con=engine)
wave_files_i = files[files["Type"]=="spec"]["Name"].to_list()
wave_files = files[files["Type"]=="spec"]

wave_data = pd.DataFrame([])
url = "https://www.ndbc.noaa.gov/data/realtime2/"
for file_name in wave_files_i: 
    file=wave_files[wave_files["Name"]==file_name]
    data_url= rf"{url}{file_name}"
    print(data_url)
    data= pd.read_csv(data_url, sep=r'\s+', header=[0,1])

    data.columns = data.columns.map(' : '.join)

    data["FileName"]=file_name
    data["Last modified"]=file["Last modified"]
    data["Last modified"]=file["ID"]
    wave_data=pd.concat([data,wave_data])

https://www.ndbc.noaa.gov/data/realtime2/41002.spec
https://www.ndbc.noaa.gov/data/realtime2/41004.spec
https://www.ndbc.noaa.gov/data/realtime2/41008.spec
https://www.ndbc.noaa.gov/data/realtime2/41009.spec
https://www.ndbc.noaa.gov/data/realtime2/41010.spec
https://www.ndbc.noaa.gov/data/realtime2/41013.spec
https://www.ndbc.noaa.gov/data/realtime2/41025.spec
https://www.ndbc.noaa.gov/data/realtime2/41040.spec
https://www.ndbc.noaa.gov/data/realtime2/41043.spec
https://www.ndbc.noaa.gov/data/realtime2/41044.spec
https://www.ndbc.noaa.gov/data/realtime2/41047.spec
https://www.ndbc.noaa.gov/data/realtime2/41049.spec
https://www.ndbc.noaa.gov/data/realtime2/41052.spec
https://www.ndbc.noaa.gov/data/realtime2/41053.spec
https://www.ndbc.noaa.gov/data/realtime2/41056.spec
https://www.ndbc.noaa.gov/data/realtime2/41065.spec
https://www.ndbc.noaa.gov/data/realtime2/41067.spec
https://www.ndbc.noaa.gov/data/realtime2/41070.spec
https://www.ndbc.noaa.gov/data/realtime2/41076.spec
https://www.

In [36]:
wave_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 542692 entries, 0 to 2152
Data columns (total 3 columns):
 #   Column                                                                 Non-Null Count   Dtype 
---  ------                                                                 --------------   ----- 
 0   #YY  MM DD hh mm WVHT  SwH  SwP  WWH  WWP SwD WWD  STEEPNESS  APD MWD  542692 non-null  object
 1   FileName                                                               542692 non-null  object
 2   Last modified                                                          213 non-null     object
dtypes: object(3)
memory usage: 16.6+ MB


In [55]:
wave_test = pd.read_csv("https://www.ndbc.noaa.gov/data/realtime2/41002.spec", sep='\s+', header=[0,1])

In [58]:
wave_test.columns = wave_test.columns.map(' : '.join)
wave_test

,#YY : #yr,MM : mo,DD : dy,hh : hr,mm : mn,WVHT : m,SwH : m,SwP : sec,WWH : m,WWP : sec,SwD : -,WWD : degT,STEEPNESS : -,APD : sec,MWD : degT
0,2024,11,20,1,10,1.1,1.0,12.1,0.3,4.0,NE,NW,SWELL,7.1,46
1,2024,11,20,0,40,1.3,1.2,12.9,0.3,4.0,NE,NW,SWELL,7.5,49
2,2024,11,20,0,10,1.2,1.1,11.4,0.3,4.0,NE,WNW,SWELL,7.2,55
3,2024,11,19,23,40,1.2,1.2,12.9,0.3,3.8,NE,NW,SWELL,7.2,40
4,2024,11,19,23,10,1.2,1.2,12.1,0.3,3.7,NE,NW,SWELL,7.1,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,2024,10,6,2,40,1.9,1.9,11.4,0.5,4.8,ENE,NNE,SWELL,8.2,71
2149,2024,10,6,2,10,1.9,1.9,10.8,0.3,3.7,ENE,NNE,SWELL,8.2,74
2150,2024,10,6,1,10,1.7,1.6,12.9,0.3,3.8,E,NNE,SWELL,7.8,96
2151,2024,10,6,0,40,1.8,1.8,11.4,0.4,4.5,ENE,NE,SWELL,8.1,71


In [54]:
first_row = data.iloc[0].tolist()
column_names=data.columns.astype(str)

data.columns = f"{data.columns.astype(str)} ({data.iloc[0]})"

TypeError: Index(...) must be called with a collection of some kind, "Index(['#YY  MM DD hh mm WVHT  SwH  SwP  WWH  WWP SwD WWD  STEEPNESS  APD MWD',\n       'FileName', 'Last modified'],\n      dtype='object') (#YY  MM DD hh mm WVHT  SwH  SwP  WWH  WWP SwD WWD  STEEPNESS  APD MWD    #yr  mo dy hr mn    m    m  sec    m  sec  -  ...\nFileName                                                                                                        SSBN7.spec\nLast modified                                                                                                          NaN\nName: 0, dtype: object)" was passed